# Динамика показателей канала дома и на даче
Пример расчета показателей телеканалов в разбивке по месту просмотра.

## Описание задачи и условий расчета
- Период: 01.01.2022 - 31.03.2022 по месяцам + тотал за период
- Временной интервал: 05:00-29:00
- ЦА: Все 14-44
- Место просмотра: Все места с разбивкой на места
- Каналы: ТНТ									
- Статистики: Share, Rtg(000)

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телесетей получим id нужных нам каналов
cats.get_tv_net(name=["ТНТ"])

# Это id 83

In [ ]:
# В справочнике демографических атрибутов найдем необходимые нам переменные
cats.get_tv_demo_attribute(names=['возраст'])

# Для формирования условий запроса нам понадобятся значения столбца entityName: age

In [ ]:
# В справочнике мест просмотра получим id мест
cats.get_tv_location()

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-01-01', '2022-03-31')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал
time_filter = None

# Задаем ЦА: Все 14-44
basedemo_filter = 'age >= 14 AND age <= 44'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы: ТНТ
company_filter = 'tvNetId IN (83)'

# Указываем список статистик для расчета
statistics = ['Share', 'Rtg000']

# Указываем срезы: месяц
slices = ['researchMonth']

# Задаем условия сортировки
sortings = None

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
    "totalType": "TotalChannels" #база расчета Share: Total Channels. Возможны опции: TotalTVSet, TotalChannelsThem
}

Сформируем словарь мест просмотра

In [ ]:
locs = {
    "Все места": None, #также можно задать через 'locationId IN (1,2)'
    "Дом": "locationId = 1",
    "Дача": "locationId = 2"
}

## Расчет задания

### Разбивка по месяцам

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждого места просмотра формируем задание и отправляем на расчет
for loc_name, expression in locs.items():
    
    # Подставляем значения словаря в параметры
    project_name = loc_name 
    location_filter = expression
          
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df_months = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df_months = df_months[['prj_name']+slices+statistics]

df_months

### За весь период

In [ ]:
# уберем разбивку по месяцам - оставим параметр slices пустым
slices = []

Прогоним цикл еще раз

In [ ]:
# Посчитаем задания в цикле
tasks = []
print("Отправляем задания на расчет")

# Для каждого места просмотра формируем задание и отправляем на расчет
for loc_name, expression in locs.items():
    
    # Подставляем значения словаря в параметры
    project_name = loc_name 
    location_filter = expression
          
    # Формируем задание для API TV Index в формате JSON
    task_json = mtask.build_timeband_task(task_name=project_name, date_filter=date_filter, 
                                          weekday_filter=weekday_filter, daytype_filter=daytype_filter, 
                                          company_filter=company_filter, time_filter=time_filter, 
                                          basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter,
                                          location_filter=location_filter, slices=slices, 
                                          statistics=statistics, sortings=sortings, options=options)

    # Для каждого этапа цикла формируем словарь с параметрами и отправленным заданием на расчет
    tsk = {}
    tsk['project_name'] = project_name    
    tsk['task'] = mtask.send_timeband_task(task_json)
    tasks.append(tsk)
    time.sleep(2)
    print('.', end = '')
    
print(f"\nid: {[i['task']['taskId'] for i in tasks]}") 

print('')
# Ждем выполнения
print('Ждем выполнения')
tsks = mtask.wait_task(tasks)
print('Расчет завершен, получаем результат')

# Получаем результат
results = []
print('Собираем таблицу')
for t in tasks:
    tsk = t['task'] 
    df_result = mtask.result2table(mtask.get_result(tsk), project_name = t['project_name'])        
    results.append(df_result)
    print('.', end = '')
df_total = pd.concat(results)

# Приводим порядок столбцов в соответствие с условиями расчета
df_total = df_total[['prj_name']+slices+statistics]

df_total

In [ ]:
# добавим колонку с периодом
df_total.insert(1, 'researchMonth', 'total_period')
df_total

### Объединяем таблицы

In [ ]:
df = pd.concat([df_months, df_total])
df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - периоды, столбцы - места просмотра, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['researchMonth'], 
                        columns=['prj_name'])
df

In [ ]:
# Поменять местами уровни в столбцах (раскомментируйте при необходимости)
#df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

#df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('03_locations_by_months')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)